In [ ]:
!pip install python-dotenv openai SpeechRecognition ffmpeg-python >> /dev/null

Load OpenAI API key which is stored in .env file.

The content should be in the format: OPENAI_API_KEY=sk-abcd12345xxxxxxxxx

In [ ]:
from dotenv import load_dotenv, find_dotenv, dotenv_values
import openai
import os

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Access the OpenAI API key
openai.api_key = os.environ["OPENAI_API_KEY"]

1. Format the prompt and specify the 'system' role;
2. Record the chat history in 'assitant' role;
3. Setting the end condition of the chat.

In [ ]:
import re
import ast

delimiter = "####"

# menu = {
#     "red bean soup": 7.25,
#     "grass jelly soup": 6.75,
#     "almond soup": 7.25,
#     "taro ball": 7.25,
#     "grass jelly": 7.25
# }

system_content = '''
You are a restaurant chatbot, the restaurant name is Meetfresh.
There is no delievery service, you don't need to ask the information like phone number or address.
When the guest finish the order, output the order details as a list of Python objects in the following format:
{'item':  'name of the ordered item', 'count': <number of ordered items>, 'unit price': <price of this item>}
Add #### before the first element and after the last element.

I can help you with our menu and take your order.
Here are the menu:

Hot Dessert:
Red Bean Soup, Price: 7.25
Grass Jelly Soup, Price: 6.75
Almond Soup, Price: 7.25

Icy Dessert:
Taro Ball, Price: 7.25
Grass Jelly, Price: 7.25

Shaved Ice:
Red Bean Milk Shaved Ice: 12.5
Mango Milk Shaved Ice: 13.5
Strawberry Milk Shaved Ice: 13.5

Tofu Pudding (available for both hot and cold):
Red Bean Tofu Pudding: 7.75
Peanut Tofu Pudding: 7.75
Rice Ball Tofu Pudding: 7.75
Hot Almond Tofu Pudding: 8.75

Cold Drink:
Black Tea: 4.5
Jasmine Green Tea: 4.5
Grass Jelly Milk Tea: 5.75
Pudding Milk Tea: 6.25
Black Sugar Boba Fresh Milk: 6.5

Please let me know if you would like to order anything or if you have any questions.
'''
messages = [
    {"role": "system", "content": system_content}
]

# Parameters of llm
model, temperature, max_tokens = "gpt-3.5-turbo", 0, 500


# Define a dictionary to store the order
guest_order = {
    "items": [],
    "total_price": 0
}
orders = []

def chat_with_bot(user_message):
    global messages
    global orders

    messages.append({"role": "user", "content": user_message})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    bot_response = response['choices'][0]['message']['content']
    messages.append({"role": "assistant", "content": bot_response})

#     # Check if the user message indicates an order
#     order_match = re.search(r'order\s+(.*?)(?:\.|\Z)', user_message, re.IGNORECASE)
#     if order_match:
#         item = order_match.group(1).strip().lower()

#         # Remove leading articles from the item
#         articles = ["a", "an", "the"]
#         for article in articles:
#             if item.startswith(article + " "):
#                 item = item[len(article) + 1 :].strip()

#         # Extract the item from the user message
#         if item in menu:
# #             print(item)
#             price = menu[item]
#             guest_order["items"].append({"item": item.capitalize(), "price": price})
#             guest_order["total_price"] += price
# #             print(guest_order)
#         else:
#             bot_response = "I'm sorry, but that item is not available. Please choose from the menu."
#     else:
#         item = ""  # Set a default value for item

    # Parse order information from bot's response
#     order_objects = []
#     order_blocks = re.findall(r'####\n(.*?)####', bot_response, re.DOTALL)
#     for order_block in order_blocks:
#         order_lines = order_block.strip().split('****')
#         for order_line in order_lines:
#             order_dict = ast.literal_eval(order_line.strip())
#             order_objects.append(order_dict)
#     if len(order_objects) > len(orders):
#         orders = order_objects

    order_objects = []
    order_block = re.search(r'####\n(.*?)####', bot_response, re.DOTALL)
    if order_block:
        order_lines = order_block.group(1).strip().split('\n')
        for order_line in order_lines:
            order_dict = ast.literal_eval(order_line.strip())
            order_objects.append(order_dict)
    if len(order_objects) > len(orders):
        orders = order_objects

    return bot_response

# while True:
#     user_input = input("You: ")
#     bot_response = chat_with_bot(user_input)
#     print("Bot:", bot_response)

#     if user_input.lower() == "bye":
#         break


Sample dialogue:
```
You: Hello!
Bot: Hello! Welcome to our restaurant. May I have your name, please?
You: alex
Bot: Hello Alex! How can I assist you today?
You: Can you show me the menu?
Bot: Of course, Alex! Here are the available desserts on our menu:

Hot Dessert:
1. Red Bean Soup - Price: $7.25
2. Grass Jelly Soup - Price: $6.75
3. Almond Soup - Price: $7.25

Icy Dessert:
1. Taro Ball - Price: $7.25
2. Grass Jelly - Price: $7.25

Please let me know if you would like to order anything or if you have any questions.
You: I'd like to have taro ball
Bot: Great choice, Alex! How many Taro Balls would you like to order?
You: two
Bot: Got it, Alex! You would like to order 2 Taro Balls. The unit price for Taro Ball is $7.25. Anything else you would like to order?
You: I also want to have almond soup
Bot: Sure, Alex! How many bowls of Almond Soup would you like to order?
You: 1
Bot: Understood, Alex! You would like to order 1 bowl of Almond Soup. The unit price for Almond Soup is $7.25.

Here is the summary of your order:

####
{'item': 'Taro Ball', 'count': 2, 'unit price': 7.25}
{'item': 'Almond Soup', 'count': 1, 'unit price': 7.25}
####

Please let me know if there is anything else I can assist you with.
You: that's all
Bot: Alright, Alex! I have noted down your order. Here is the summary:

####
{'item': 'Taro Ball', 'count': 2, 'unit price': 7.25}
{'item': 'Almond Soup', 'count': 1, 'unit price': 7.25}
####

Thank you for placing your order! If you have any further questions or need assistance, feel free to ask.
You: bye
Bot: Goodbye, Alex! Have a great day and enjoy your desserts. If you have any more questions in the future, feel free to ask. Take care!
```

In [ ]:
for item in orders:
  print(item)

{'item': 'Red Bean Soup', 'count': 2, 'unit price': 7.25}
{'item': 'Almond Soup', 'count': 2, 'unit price': 7.25}


Audio recording: embed a HTML and using JavaScript code to access local microphone

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import scipy
import speech_recognition

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
# test the recording, audio is a numpy array
audio, sr = get_audio()

In [ ]:
# Initiate a speech recognizer instance
recognizer = speech_recognition.Recognizer()

# The file path of the recording, this path is working for Google colab
AUDIO_FILE = ("/content/recording.wav")

In [ ]:
# write the audio, which ia numpy arry, into a '.wav' file
scipy.io.wavfile.write('recording.wav', sr, audio)

In [ ]:
# Test the speech recognizer
with speech_recognition.AudioFile(AUDIO_FILE) as source:
    #reads the audio file. Here we use record instead of
    #listen
    audio = recognizer.record(source)
recognizer.recognize_google(audio)


'hello'

In [ ]:
# Test for a loop of speech recognition. The loop print the user speech and quits when user says 'bye'.
while True:
  audio, sr = get_audio()
  scipy.io.wavfile.write('recording.wav', sr, audio)
  with speech_recognition.AudioFile(AUDIO_FILE) as source:
    audio = recognizer.record(source)
  speech = recognizer.recognize_google(audio)
  print(speech)
  if speech == 'bye':
    break

hello


bye


In [ ]:
# Text version
messages = [
    {"role": "system", "content": system_content}
]
orders = []
while True:
    user_input = input("You: ")
    bot_response = chat_with_bot(user_input)
    print("Bot:", bot_response)

    if user_input.lower() == "bye":
        break


You: hi
Bot: Hello! How can I assist you today?
You: can you show me the menu
Bot: Of course! Here is our menu:

Hot Dessert:
1. Red Bean Soup - Price: $7.25
2. Grass Jelly Soup - Price: $6.75
3. Almond Soup - Price: $7.25

Icy Dessert:
4. Taro Ball - Price: $7.25
5. Grass Jelly - Price: $7.25

Shaved Ice:
6. Red Bean Milk Shaved Ice - Price: $12.5
7. Mango Milk Shaved Ice - Price: $13.5
8. Strawberry Milk Shaved Ice - Price: $13.5

Tofu Pudding (available for both hot and cold):
9. Red Bean Tofu Pudding - Price: $7.75
10. Peanut Tofu Pudding - Price: $7.75
11. Rice Ball Tofu Pudding - Price: $7.75
12. Hot Almond Tofu Pudding - Price: $8.75

Cold Drink:
13. Black Tea - Price: $4.5
14. Jasmine Green Tea - Price: $4.5
15. Grass Jelly Milk Tea - Price: $5.75
16. Pudding Milk Tea - Price: $6.25
17. Black Sugar Boba Fresh Milk - Price: $6.5

Please let me know if you would like to order anything or if you have any questions.
You: what is grass jelly
Bot: Grass jelly is a popular ingredient 

In [ ]:
# Voice version
messages = [
    {"role": "system", "content": system_content}
]
orders = []
while True:
  audio, sr = get_audio()
  scipy.io.wavfile.write('recording.wav', sr, audio)
  with speech_recognition.AudioFile(AUDIO_FILE) as source:
    audio = recognizer.record(source)
  user_input = recognizer.recognize_google(audio)
  print('You: ' + user_input)
  bot_response = chat_with_bot(user_input)
  print("Bot:", bot_response)

  if user_input.lower() == "bye":
    break


You: hello
Bot: Hello! Welcome to Meetfresh. How can I assist you today?


You: can you show me the phone menu
Bot: Sure! Here is our menu:

Hot Dessert:
1. Red Bean Soup - Price: $7.25
2. Grass Jelly Soup - Price: $6.75
3. Almond Soup - Price: $7.25

Icy Dessert:
4. Taro Ball - Price: $7.25
5. Grass Jelly - Price: $7.25

Shaved Ice:
6. Red Bean Milk Shaved Ice - Price: $12.5
7. Mango Milk Shaved Ice - Price: $13.5
8. Strawberry Milk Shaved Ice - Price: $13.5

Tofu Pudding (available for both hot and cold):
9. Red Bean Tofu Pudding - Price: $7.75
10. Peanut Tofu Pudding - Price: $7.75
11. Rice Ball Tofu Pudding - Price: $7.75
12. Hot Almond Tofu Pudding - Price: $8.75

Cold Drink:
13. Black Tea - Price: $4.5
14. Jasmine Green Tea - Price: $4.5
15. Grass Jelly Milk Tea - Price: $5.75
16. Pudding Milk Tea - Price: $6.25
17. Black Sugar Boba Fresh Milk - Price: $6.5

Please let me know if you would like to order anything or if you have any questions.


You: I'd like to order the rabbit being soup
Bot: Great choice! How many bowls of Red Bean Soup would you like to order?


You: just the one
Bot: Alright! You have ordered 1 bowl of Red Bean Soup. The unit price for Red Bean Soup is $7.25. 

#### 
{'item': 'Red Bean Soup', 'count': 1, 'unit price': 7.25}
#### 

Is there anything else you would like to order?


You: I also want to try the peanut tofu pudding
Bot: Sure! How many bowls of Peanut Tofu Pudding would you like to order?


You: one please
Bot: Great! You have ordered 1 bowl of Peanut Tofu Pudding. The unit price for Peanut Tofu Pudding is $7.75.

#### 
{'item': 'Peanut Tofu Pudding', 'count': 1, 'unit price': 7.75}
#### 

Is there anything else you would like to order?


You: that's all I want
Bot: Alright! Here is your order summary:

####
{'item': 'Red Bean Soup', 'count': 1, 'unit price': 7.25}
{'item': 'Peanut Tofu Pudding', 'count': 1, 'unit price': 7.75}
####

Thank you for your order! If you have any other questions, feel free to ask.


You: bye
Bot: Goodbye! Have a great day!


In [ ]:
for item in orders:
  print(item)

{'item': 'Red Bean Soup', 'count': 1, 'unit price': 7.25}
{'item': 'Peanut Tofu Pudding', 'count': 1, 'unit price': 7.75}
